## NNUE (Efficiently Updatable Neural Network)

I am going to build and train an NNUE, NNUEs are impletemented in chess engines because of their speed. The idea behind them is that whenever a move is made in an already evaluated position, the position doesn't change that much. Hence the data used to represent it doesn't change much either, which means (depending on the network structure) that some operations inside the neural network made to evaluate the current position are the same as for the previous position. NNUE's take advantage of this fact, by only making the operations that will be different. To do this we must store information of how the NNUE worked on the previous position, we do this by means of an accumulator. 

My idea is to build and train a neural network with Tensorflow, after training I will get the weights of the NN and create an identical NN without the first layer. The output of the first layer will be substituted with accumulators, these will be shorter (maybe length 16) than the original input. And they will be easily updated and later fed into the new NN.


In [1]:
pip install zstandard

Note: you may need to restart the kernel to use updated packages.


In [2]:
 pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow-io-gcs-filesystem>=0.23.1 (from tensorflow) (from versions: none)
ERROR: No matching distribution found for tensorflow-io-gcs-filesystem>=0.23.1
Note: you may need to restart the kernel to use updated packages.


In [141]:
# Changing pgns to dictionaries of fens : evaluations

import chess.pgn
import re
import io 

def pgneval_to_dict(pgn):
    pgn_filetxt0 = open(pgn)
    pgn_text0 = pgn_filetxt0.read()
    evaluations0 = re.findall(r'{(.*?)}', pgn_text0)
    pgn_file0 = open(pgn)
    game0 = chess.pgn.read_game(pgn_file0)
    board0 = game0.board()
    TrainDict = {}
    for i, move in enumerate(game0.mainline_moves()):
        board0.push(move)
        fen = board0.fen()
        klk = evaluations0[i].split('/')
        if klk[0][1] not in ['1','2','3','4','5','6','7','8','9','0']:
            continue # Sometimes evaluation is mate in something
        if evaluations0[i] == 'book':
            TrainDict[fen] = '0'
        else:
            TrainDict[fen] = klk[0]
    return TrainDict


def pgneval_to_dict(pgn_file):
    pgn_filetxt = open(pgn_file)
    pgn_text = pgn_filetxt.read()
    games = re.findall(r'\n\n(.*?)\n\n', pgn_text, re.DOTALL)
    TrainDict = {}
    for game_text in games:
        game = chess.pgn.read_game(io.StringIO(game_text))
        board = game.board()
        evaluations = re.findall(r'{(.*?)}', game_text)
        for i, move in enumerate(game.mainline_moves()):
            board.push(move)
            fen = board.fen()
            klk = evaluations[i].split('/')
            if klk[0][1] not in ['1','2','3','4','5','6','7','8','9','0']:
                continue # Sometimes evaluation is mate in something
            if evaluations[i] == 'book':
                TrainDict[fen] = '0'
            else:
                TrainDict[fen] = klk[0]
    return TrainDict
pgneval_to_dict('Chess pgns/t1.pgn')
# I HAVE CHECKED THAT THIS ONE IS RUNNING CORRECTLY

{'rnbqkb1r/ppp2ppp/3ppn2/8/2PP4/5NP1/PP2PP1P/RNBQKB1R b KQkq - 0 4': '+0.27',
 'rnbqk2r/ppp1bppp/3ppn2/8/2PP4/5NP1/PP2PP1P/RNBQKB1R w KQkq - 1 5': '-0.80',
 'rnbqk2r/ppp1bppp/3ppn2/8/2PP4/5NP1/PP2PPBP/RNBQK2R b KQkq - 2 5': '+0.27',
 'rnbqk2r/ppp1bppp/4pn2/3p4/2PP4/5NP1/PP2PPBP/RNBQK2R w KQkq - 0 6': '-0.91',
 'rnbqk2r/ppp1bppp/4pn2/3p4/2PP4/5NP1/PP2PPBP/RNBQ1RK1 b kq - 1 6': '+0.29',
 'rnbq1rk1/ppp1bppp/4pn2/3p4/2PP4/5NP1/PP2PPBP/RNBQ1RK1 w - - 2 7': '-0.28',
 'rnbq1rk1/ppp1bppp/4pn2/3p4/2PP4/5NP1/PPQ1PPBP/RNB2RK1 b - - 3 7': '+0.92',
 'rnbq1rk1/1pp1bppp/4pn2/p2p4/2PP4/5NP1/PPQ1PPBP/RNB2RK1 w - - 0 8': '-0.29',
 'rnbq1rk1/1pp1bppp/4pn2/p2p4/2PP4/5NP1/PPQ1PPBP/RNBR2K1 b - - 1 8': '+0.87',
 'r1bq1rk1/1ppnbppp/4pn2/p2p4/2PP4/5NP1/PPQ1PPBP/RNBR2K1 w - - 2 9': '-0.30',
 'r1bq1rk1/1ppnbppp/4pn2/p2p4/2PP4/2N2NP1/PPQ1PPBP/R1BR2K1 b - - 3 9': '+0.81',
 'r1bq1rk1/1ppnbppp/4pn2/p7/2pP4/2N2NP1/PPQ1PPBP/R1BR2K1 w - - 0 10': '-0.78',
 'r1bq1rk1/1ppnbppp/4pn2/p3N3/2pP4/2N3P1/PPQ1PPBP/R1BR2K1 b - - 1

In [142]:
# Importing the data
import os
List_pgns_train = []
for filename in os.listdir('chess pgns'):
    f = os.path.join('chess pgns', filename)
    # checking if it is a file
    if os.path.isfile(f):
        List_pgns_train.append(pgneval_to_dict(f))
        
print(List_pgns_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [143]:
# Separating the fens and the evaluations

Y_train = []
fens_train_list = []
for i in List_pgns_train:
    Y_train.append(list(i.values()))
    fens_train_list.append(list(i.keys()))
Y_train = [item for sublist in Y_train for item in sublist]
fens_train_list = [item for sublist in fens_train_list for item in sublist]
for i, value in enumerate(Y_train):
    Y_train[i]=float(value)
print(len(fens_train_list))
print(len(Y_train))

127465
127465


In [126]:
# This function will take a fen and encode the position into two arrays of length 6x64 = 384.
# Each of these arrays consists of 6 length 64 arrays concatenated, where the length 64 arrays
# represent the positions of (black or white) king, queens, rooks, bishops, knights and pawns.
import numpy as np
def encode_position(fen):
    # Define starting position FEN string
    # Define arrays for the different matrices
    white_pawns = np.zeros(64)
    white_rooks = np.zeros(64)
    white_knights = np.zeros(64)
    white_bishops = np.zeros(64)
    white_queens = np.zeros(64)
    white_king = np.zeros(64)
    black_pawns = np.zeros(64)
    black_rooks = np.zeros(64)
    black_knights = np.zeros(64)
    black_bishops = np.zeros(64)
    black_queens = np.zeros(64)
    black_king = np.zeros(64)
    # Initialize counters for the 50 moves rule and the repetition counter
    # Split FEN string into its components
    fen_parts = fen.split(' ')
    pieces = fen_parts[0]
    # Convert pieces component to a matrix representation
    rows = pieces.split('/')
    for i, row in enumerate(rows):
        file_num = 0
        for char in row:
            if char.isnumeric():
                file_num += int(char)-1
            else:
                # Set the appropriate matrix to 1 for the current square and piece
                if char == 'p':
                    black_pawns[i*8+ file_num] = 1
                elif char == 'r':
                    black_rooks[i*8+ file_num] = 1
                elif char == 'n':
                    black_knights[i*8+ file_num] = 1
                elif char == 'b':
                    black_bishops[i*8+ file_num] = 1
                elif char == 'q':
                    black_queens[i*8+ file_num] = 1
                elif char == 'k':
                    black_king[i*8+ file_num] = 1
                elif char == 'P':
                    white_pawns[i*8+ file_num] = 1
                elif char == 'R':
                    white_rooks[i*8+ file_num] = 1
                elif char == 'N':
                    white_knights[i*8+ file_num] = 1
                elif char == 'B':
                    white_bishops[i*8+ file_num] = 1
                elif char == 'Q':
                    white_queens[i*8+ file_num] = 1
                elif char == 'K':
                    white_king[i*8+ file_num] = 1
            file_num += 1
    # Return all the matrices
    listwhite = [np.flipud(white_king), np.flipud(white_queens), np.flipud(white_rooks), 
                 np.flipud(white_bishops), np.flipud(white_knights), np.flipud(white_pawns)]
    listblack = [np.flipud(black_king), np.flipud(black_queens), np.flipud(black_rooks),
                 np.flipud(black_bishops), np.flipud(black_knights), np.flipud(black_pawns)]
    return np.concatenate(listwhite, axis =0).tolist(), np.concatenate(listblack, axis =0).tolist()
# This is working correctly

In [127]:
len(encode_position('rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR b KQkq - 0 1')[0])

384

In [128]:
encode_position('rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR b KQkq - 0 1')[1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [136]:
X_train1 = []
for fen in fens_train_list:
    X_train1.append(encode_position(fen)[0])
X_train2 = []
for fen in fens_train_list:
    X_train2.append(encode_position(fen)[1])
X_train1 = np.array(X_train1)
X_train2 = np.array(X_train2)

In [47]:
import torch

# Define the custom module subclass
class MyNet(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Define the layers
        self.layer1_1 = torch.nn.Linear(384, 16, bias=True)
        self.layer1_2 = torch.nn.Linear(384, 16, bias=True)
        self.fc_layer = torch.nn.Linear(32, 16, bias=True)
        self.output_layer = torch.nn.Linear(16, 1, bias=True)
        
        # Initialize the layers with integer parameters
        self.layer1_1.weight.data = torch.randint(low=-300, high=300, size=(16, 384), dtype=torch.float16)
        self.layer1_1.bias.data = torch.randint(low=-300, high=300, size=(16,), dtype=torch.float16)
        self.layer1_2.weight.data = torch.randint(low=-300, high=300, size=(16, 384), dtype=torch.float16)
        self.layer1_2.bias.data = torch.randint(low=-300, high=300, size=(16,), dtype=torch.float16)
        self.fc_layer.weight.data = torch.randint(low=-300, high=300, size=(16, 32), dtype=torch.float16)
        self.fc_layer.bias.data = torch.randint(low=-300, high=300, size=(16,), dtype=torch.float16)
        self.output_layer.weight.data = torch.randint(low=-300, high=300, size=(1, 16), dtype=torch.float16)
        self.output_layer.bias.data = torch.randint(low=-300, high=300, size=(1,), dtype=torch.float16)
    
    def forward(self, input_1, input_2):
        # Apply the layers to the inputs
        hidden_1 = torch.nn.functional.relu(self.layer1_1(input_1))
        hidden_2 = torch.nn.functional.relu(self.layer1_2(input_2))
        concatenation = torch.cat((hidden_1, hidden_2), dim=1)
        fc_output = torch.nn.functional.relu(self.fc_layer(concatenation))
        output = torch.nn.functional.sigmoid(self.output_layer(fc_output))
        return output

In [70]:
len(X_train)

127465

### Using PyTorch

In [155]:
import torch.nn as nn

### Using Tensorflow

In [115]:
import tensorflow as tf
import numpy as np

# Define the input shape
input_shape = (384,)

# Define the input layers
input_1 = tf.keras.layers.Input(shape=input_shape)
input_2 = tf.keras.layers.Input(shape=input_shape)

# Define the first set of layers
hidden_1 = tf.keras.layers.Dense(16, activation='relu', dtype=tf.float16)(input_1)
hidden_2 = tf.keras.layers.Dense(16, activation='relu', dtype=tf.float16)(input_2)

# Define the concatenation layer
concatenation = tf.keras.layers.Concatenate()([hidden_1, hidden_2])

# Define the fully connected layer
fc_layer = tf.keras.layers.Dense(16, activation='relu', dtype=tf.float16)(concatenation)

# Define the final output layer
output = tf.keras.layers.Dense(1, activation='sigmoid', dtype=tf.float16)(fc_layer)

# Define the model
model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 384)]        0           []                               
                                                                                                  
 input_13 (InputLayer)          [(None, 384)]        0           []                               
                                                                                                  
 dense_24 (Dense)               (None, 16)           6160        ['input_12[0][0]']               
                                                                                                  
 dense_25 (Dense)               (None, 16)           6160        ['input_13[0][0]']               
                                                                                            

In [156]:
import sklearn
from sklearn.model_selection import train_test_split
X_train11, X_val11, X_train22, X_val22, Y_train0, Y_val0 = train_test_split(X_train1, X_train2, np.array(Y_train), test_size = 0.1, random_state=2)

In [152]:
matrix = model.fit([X_train11, X_train22], Y_train0, validation_data = ([X_val11, X_val22], Y_val0), epochs = 5, batch_size = 1)


Epoch 1/5
114718/114718 [==============================] - 43s 372us/step - loss: nan - accuracy: 8.5427e-04 - val_loss: nan - val_accuracy: 8.6295e-04
Epoch 2/5
114718/114718 [==============================] - 43s 373us/step - loss: nan - accuracy: 8.5427e-04 - val_loss: nan - val_accuracy: 8.6295e-04
Epoch 3/5
114718/114718 [==============================] - 43s 373us/step - loss: nan - accuracy: 8.5427e-04 - val_loss: nan - val_accuracy: 8.6295e-04
Epoch 4/5
114718/114718 [==============================] - 43s 371us/step - loss: nan - accuracy: 8.5427e-04 - val_loss: nan - val_accuracy: 8.6295e-04
Epoch 5/5
114718/114718 [==============================] - 43s 376us/step - loss: nan - accuracy: 8.5427e-04 - val_loss: nan - val_accuracy: 8.6295e-04
